# Creating subcategory_dictionary 
- same them into csv files.

In [1]:
import numpy as np
import pandas as pd
import re
import math
import unicodedata
from lib import utility as util

In [2]:
# file paths
category_file = "../raw_data/商品分類表_180410.xlsx"

# for category data
category_df = pd.read_excel(category_file,  sheet_name='Sheet1', index_col=0 )
category_df.columns = ['product_name',
                       'material_type_id',
                       'material_type',
                       'manufacture_country_id',
                       'manufacture_country',
                       'category_id',
                       'category']
category_df.index.name = 'product_id'
category_df.head(5)

,product_name,material_type_id,material_type,manufacture_country_id,manufacture_country,category_id,category
product_id,,,,,,,
AAH080-C,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-E,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-H,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-J,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー
AAH080-K,アルファベットチャームＡＤ,0.0,その他,13.0,国内仕入,60.0,アクセサリー


In [3]:
query_keywords = [
    'グラデーションスカート',
    'スカート',
    'ウッドプレート',
    'ネックレス',
    'ｈａｔ',
    'ハット',
    'キャニスター',
    'テラコッタ',
    'スリッポン',
    'スタンドミラー',
    'ピアス',
    'バッグ',
    'ｂａｇ',
    'スリッパ',
    'グローブ',
    'キャップ',
    'ｃａｐ',
    '携帯ストラップ',
    'ベッドスプレッド',
    'ハンカチ',
    'リサイクル缶',
    'ランジェ',
    'バスケット',
    'ベルト',
    'ボトルネック',
    'スヌード',
    'カーテン',
    'カッティングボード',
    'キャンドル',
    'ポーチ',
    'バケツ',
    'マフラー',
    'ケープショール',
    'ロザリオ',
    'ピッチャー',
    'ウェッジサンダル',
    'ディッシュクロス',
    'ヘアーバンド',
    'ショートパンツ',
    'マット',
    'テーブルクロス',
    'ソックス',
    'ワンピース',
    'アームウォーマー',
    'ドロワー',
    'ヘアークリップ',
    'バレエシューズ',
    'ミラー',
    'フレーム',
    'ストール',
    'ピンクッション',
    'ホルダー',
    'ミトン',
    'ヘアーピン',
    'ｊａｒ',
    'コースター',
    'ケース',
    'ブレスレット',
    'バーハンガー',
    'スカーフピン',
    'レッグウォーマー',
    'ショートポンチョ',
    'ヴェール',
    'ブーツルームシューズ',
    'チキンウィーブ',
    'クリーパーランプ',
    'ランプシェード',
    'キャニスター',
    'セーター',
    'サーブスプーン',
    'キッチンパッド',
    'ブラウス',
    'ワードローブ',
    'バレッタ',
    'テーブルセンター',
    'チュニック',
    'プレート',
    'パフューム',
    'ルームシューズ',
    'コンチャ',
    'リング',
    'カードスタンド',
    'ディスプレイカバー',
    'クッションカバー',
    'ガーデンポット',
    'ニットピン',
    'フルエプロン',
    'ディンオープントゥシューズ',
    'バングル',
    'ブルクロス',
    'プランプイアー',
    'オーナメント',
    'リュースガラス',
    'リューズガラス',
    'カーディガン',
    'タンクトップ',
    '３連ブレス',
    'ストラップ',
    'フラワーボウル',
    '腕時計',
    'クルタシャツ',
    'デッキシューズ',
    'フォーク',
    'ドアノブ',
    'ソープディッシュ',
    'フラックス',
    'ヘアーゴム',
    '和しらべスプーン',
    'エプロン',
    'スタンド',
    'ショート',
    'ウォレット',
    'スリープピン',
    'タブレットトレイ',
    'キッチンラック',
    'ウォーターポット',
    'ヨガドレス',
    'ホールドピン',
    'フォローラインパンツ',
    'モンブランヘアゴム',
    'シュシュ',
    'マグネット',
    'ｋｅｙチャーム',
    'ディスプレイケージ',
    'ヘアゴム',
    'キッチンクロス',
    '時計',
    'ツウィルシューズ',
    'サンシェード',
    'ソープデッシュ',
    'デイリーウェア',
    'レギンス',
    'キルトクッション',
    'スコップ',
    'レターｂｏｘ',
    'フラットトレイ',
    'ハンガーラック',
    'スニーカー',
    'カップ',
    'チェア',
    'ｕｓａステッカー',
    'ソートトレー',
    'ウォール ハンギング',
    'ペンダントコード',
    'ポ−チ',
    'ワイヤーリボン',
    'ピローカバー',
    'キャミソール',
    '封筒',
    'ブラシ',
    'ヘアバンド',
    'うちわ',
    'ディナースプーン',
    'トレーｂｏｘ',
    'クリップ',
    'ピック',
    'ベル',
    'トレー',
    '手袋',
    'メタリック５連ブレス',
    'ブローチ',
    '扇子',
    '暖簾',
    'のれん',
    '布',
    'ライト',
    'ウッドフロー',
    'ベンチ',
    'クリーパートレリス',
    'キャスケット',
    'けずり箸',
    'ミニチャーム',
    'ブレス',
    'テーブル',
    'クッション',
    'ジャケット',
    'タオル',
    'コサージュ',
    'がま口',
    'がまぐち',
    'スカーフ',
    'ティーコーゼ',
    'ブリキ鏡',
    'ファイルボックス',
    'つば広ラフィア帽',
    'サンバイザー',
    'ドライプラント',
    'イヤーマフ',
    'パンツ',
    'プルクロス',
    'パトロンデナバホドレス',
    'ヒンジ',
    'ｇｌａｓｓ',
    'ボックス',
    'チェックドレス',
    'ブックシェルフ',
    'ガーランド',
    'ショール',
    'ディフューザー',
    'ｓｏｃｋｓ',
    'ｉｐｈｏｎｅ６／６ｓ カ  バー',
    'ｉｐｈｏｎｅ６／６Ｓ カ  バー',
    'ウォールラック',
    'クロシェ',
    'フラワーポット',
    'ポット',
    'バードハウス',
    'シューズ',
    'クロッシュバード',
    'ウォールステップラック',
    'ハンギング',
    'シャーレ',
    'グラス',
    'コンプロバールシャツ',
    'チョーカー',
    'プレーン',
    'フラットショルダー',
    'ラック',
    'カレンダー',
    'カバーウォーマー',
    'ホウキ',
    'シャツ',
    'ブランケット',
    'カバードｂｏｘ',
    'サンダル',
    '衿付きポンチョ',
    'カチューシャ',
    'アロマミスト',
    'ダストパン',
    'フリークロス',
    'コンフォートカバー',
    'オブジェ',
    'ラウンドスツール',
    'ティペット',
    'ティペッ ト',
    'ティペッ  ト',
    'カトラリーレスト',
    'ｍａｔ',
    'ティーセット',
    'ｔｅａセット',
    'プルオーバー',
    'プルオー  バー',
    'ボウル',
    'タペストリー',
    'ボトル',
    'チャーム',
    'イヤーカフ',
    'カチューム',
    'ストッカー',
    'ハンドウォーマー',
    'トート',
    'ハンガーカバー',
    'コーディガン',
    'リース',
    'スプーン',
    'ネックウォーマー',
    'ロングベスト',
    'くつべら',
    'ピン',
    'ｂｏｘ',
    'ワイヤー',
    'ベスト',
    'ヤツィドレス',
    'パッチドレス',
    'シェルフボード',
    'タンブラー',
    '防寒',
    'インセンス',
    'サイフ',
    'ダブルリム',
    'ベレー',
    'キャミソ',
    'メモカバー',
    'ペーパーハンガー',
    'エルソールドレス',
    'ハンガー',
    'ワックスコード',
    '電球コード',
    'ディナーナイフ',
    'ｆｕｒイヤーバンド',
    'キッチンセット',
    '木版',
    'ふちどり帽子',
    'ガウン',
    'アクセサリー',
    '巾着',
    '香立',
    'ｆｒａｍｅ',
    'ワゴン',
    'ローラー',
    'ホウロウ',
    'ホ−ロー',
    'つば広帽',
    '八角箸',
    'フラワー',
    'ローズマリー',
    'オリーブ',
    'タイツ',
    'サスペンダー',
    'アルファベット',
    'アンクレット',
    'バンダナ',
    'コート',
    'イヤーバンド',
    'ポンチョ',
    'リュック',
    'ランプ',
    'ドール',
    'ビーカー',
    'サロペット',
    'ペンダント',
    'スツール',
    'ボタン',
    'クレセントコーム',
    'ヒモ',
    'コロフドレス',
    'シェルフ',
    'トレイ',
    'ポシェット',
    'オーナメンタル',
    'キーカバー',
    'バスルームラック',
    'ガーゼ',
    'ラウンドハンドル',
    'トピアリー',
    'スクープ',
    'ルエダドレス',
    'レードル',
    'ボード',
    'プランニングシザー',
    'プランター',
    'ディスプレイ',
    '更紗',
    'ロザリー',
    'クレッセントコーム',
    'フラスコ',
    'ネイル',
    'アンブレラグリッパー',
    'ドローアップチェスト',
    'レッグウ  ォーマー',
    'ショルダー',
    'レスト',
    'パントーンバンドル',
    'クロック',
    'マグ',
    'サラッペ',
    'ルーテ',
    'ポストカード',
    'オイル',
    'グリップ ハンドル',
    'メモパッド カバー',
    'ｐｏｔｔｅｒｙ',
    'パース',
    'シザー',
    'ブロック',
    '福袋',
    'ラッチＫＥＹ',
    'ラッチｋｅｙ',
    'ジョーロ',
    'コーム',
    'カード',
    'ラベンダーカモミール',
    'トップス',
    'トルソー',
    'ジレ',
    'ボール',
    'クリスマス',
    '刺繍ドレ',
    'カゴ',
    'キッチンパット',
    'ルームブーツ',
    'ワンピー ス',
    'KNOB',
    'ｋｎｏｂ',
    'スタンプ',
    'ケープ',
    'レッグ',
    'フィンガーパペット',
    'サボ',
    'バインダー',
    'ホーロー',
    'リップバーム',
    'ハンチング',
    'スタッフド',
    'ベース',
    'ブーツウォーマー',
    'コンテナ',
    'ベビーパウダー',
    'ソーイングキット',
    'ドローアップチェスト',
    'ハニーコム',
    'コンパクトチェスト',
    'ブックマーク',
    'マドラー',
    'ウォール ポケット',
    'ロッカーチェスト',
    'コーヒードリッパー',
    'パイプ',
    'フルーツ',
    'ステッカー',
    'ピロー',
    'バード',
    'ベッドスプレット',
    'コンソール',
    'ウォールキャビネット',
    'ドローアップチェスト',
    'ドローアップ チェスト',
    'ストライプ',
    'ツリー',
    'アムレット',
    'ワンピー  ス',
    'ＢＡＧ',
    'パーカー',
    'ジャム',
    'ＨＡＴ',
    'レギパン',
    'リブＣＡＰ',
    'パウダー',
    'ＣＡＰ',
    '耳宛ＣＡＰ',
    'ＢＯＸ',
    'ＤＲＡＷＥＲ',
    'ＭＡＴ',
    'ＧＬＡＳＳ',
    'ＰＯＴＴＥＲＹ',
    'ドリンク',
    'ホ－ロー',
    'バタークリーム',
    'パルシュドレス',
    'ラフィア帽',
    'シューホーン',
    'ＦＲＡＭＥ',
    'ターナー',
    'ＫＮＯＢ',
    'プラントカバー',
    'スパチュラ',
    '灰皿',
    'オールインワン',
    'ジンジャーピーチ',
    'チーズ',
    'ストロベリー',
    'スワッグ',
    'インナードレス',
    'キャリーゴザ',
    'バケット',
    'Peg',
    'ペグ',
    'バターナイフ',
    'リャマ人形',
    'モチーフゴム',
    'クロス',
    'フーディ',
    'ジョーゴ',
    '子供長靴',
    '重箱',
    'ペーストナイフ',
    'むぎわら帽',
    'ブロウガラス',
    'ココナッツライム',
    'ブーケ',
    'キャビネット',
    '壁掛け',
    'ブーツ',
    'カギ',
    'カットソー',
    'ＣＡＮＤＬＥ',
    '糸',
    'ティピティピドレス',
    'カーデ',
    'チェーン',
    'バタービーター',
    'ベビーティアーズ',
    'メタルバッジ',
    'ケージ',
    'アイボルト',
    'ガイコツ',
    'ポ－チ',
    'ＳＯＣＫＳ',
    '根竹ハンドル',
    'アートタイル',
    'カバー',
    'ファイル',
    'ｉｐｈｏｎｅ６／６Ｓケー  ス',
    'ブックエンド',
    'パーツ',
    'ハウス',
    'ドイリー',
    'チャーチ',
    'ベチバー',
    'ロングＳＫ',
    'スノーフレーク',
    'ＪＡＲ',
    'キッズキャミ',
    'ＢＥＡＲ',
    'サンタ',
    'エストレジャス',
    'コンフィチュール',
    'クレイジーマンパ',
    'ＳＫ',
    '絵皿',
    'オレガノ',
    'バジル',
    'イカット',
    'スプレッド',
    'テスター',
    'セージ',
    'ユーカリ',
    'ハニーデュメロン',
    'ラベンダー',
    'プミラ',
    'サークルハイゲージ',
    'カラビナ',
    'シャカシャカ',
    'ＣＵＰ',
    'タブ',
    'レインディア',
    'エレサー',
    'ケット',
    'メンズ',
    'レディース',
    'ギャザー',
    'バングル',
    'ヒッチ',
    'カード　ラック'
]

product_names = util.convert_col_to_list(category_df['product_name'])
print("There are {} product names".format(len(product_names)))

unique_product_names = util.convert_to_unique_list(product_names)
print("There are {} unique product names".format(len(unique_product_names)))

filtered_names = util.filter_by_another_list(unique_product_names, query_keywords) 
print("{} query keywords have been found".format(len(query_keywords)))
print("{} of products still remain unassigned with design name".format(len(filtered_names)))

filtered_names[:5]

There are 21863 product names
There are 11575 unique product names
524 query keywords have been found
271 of products still remain unassigned with design name


['カラーＪＱパッチデニムＰＴ Ｍ→Ｌ ＢＬ  ',
 'アンティーク ダリ ＫＵＴＣＨ  ',
 'ＰａｙＰａｌ決済手数料  ',
 'アイアンワーク レングス “Ｓ”フック（Ｌ）  ',
 'ミッシュ ロングＢＬ      ＢＫ  ']

In [4]:
group_dict = {
    '財布': ['がまぐち','がま口','ウォレット','サイフ'],
    '帽子': ['帽','hat','キャップ','cap','ハット','hat'],
    'ドレス': ['ドレ','dress'],
    '箸' : ['箸','おはし'],
    'グリッパー': ['グリッパー'],
    'タイル': ['タイル'],
    'ウォーマー': ['ウォーマー'],
    '耳あて': ['イヤー'],
    'プレート': ['プレート'],
    'フロー': ['フロー'],
    '壁飾り': ['ウォール','壁掛け','壁飾り'],
    'バッグ': ['バッグ','bag','カゴ','ショルダー','リュック'],
    'スカート': ['sk','スカート'],
    'レスト': ['レスト'],
    'ウォーマー': ['ウォーマー'],
    'ボックス': ['box','ボックス','箱'],
    'カーディガン': ['カーデ'],
    'ラック': ['ラック','rack','カード　ラック'],
    'ポット': ['ポット'],
    'スタンド':['スタンド'],
    'キャミソール': ['キャミ'],
    'キッチン用品': ['キッチン'],
    'クッション':['クッション'],
    'カバー': ['カバー'],
    'ランプ': ['ランプ'],
    'シャツ': ['シャツ'],
    'クレセントコーム': ['クレセントコーム','クレッセントコーム'],
    'グラス': ['グラス'],
    'ハンドル': ['ハンドル'],
    'ケース': ['ケース'],
    'ショール': ['ショール'],
    'サンタクロース': ['サンタ'],
    'スプーン':['スプーン'],
    '靴': ['シューズ','靴','スリッポン','サンダル','ブーツ'],
    'ショート': ['ショート'],
    'パンツ': ['パンツ','ズボン','レギンス'],
    'スカーフ': ['スカーフ'],
    'ピン': ['ピン'],
    'スツール': ['スツール'],
    'ステッカー':['ステッカー'],
    'ストラップ': ['ストラップ','ストライプ'],
    'ソープディッシュ': ['ソープディッシュ','ソープデッシュ'],
    'テーブル': ['テーブル'],
    'クロス': ['クロス'],
    'ケージ': ['ケージ'],
    'ナイフ': ['ナイフ'],
    'トレー': ['トレー','トレイ','トレー'],
    'ピン': ['ピン'],
    'ノブ': ['knob','ノブ'],
    'ドロワー': ['ドロワー','drawer','ドロワ'],
    'ハンギング':['ハンギング'],
    'クリーム': ['クリーム'],
    'パペット': ['パペット','人形','ドール'],
    'ボウル': ['ボウル'],
    'エプロン': ['エプロン'],
    'ハサミ': ['シザー','ハサミ','scissor'],
    'クリップ': ['クリップ'],
    'ゴム': ['ゴム'],
    'ヘアーバンド': ['ヘアーバンド','ヘアバンド'],
    'ハンガー': ['ハンガー'],
    'カード': ['カード'],
    'パウダー': ['パウダー'],
    'ホルダー': ['ホルダー'],
    'マット': ['マット','mat'],
    '鏡': ['ミラー'],
    'チャーム': ['チャーム'],
    'ブレス': ['ブレス'],
    'バッジ': ['バッジ'],
    'ベスト': ['ベスト'],
    'ローブ': ['ローブ'],
    'グルーブ': ['手袋','グローブ'],
    '香立': ['アロマ','香立','インセンス','incent','インセント'],
    '時計': ['時計','クロック','腕時計','ウォッチ'],
    'pottery': ['皿','cup','カップ','コップ'],
    'ポンチョ': ['ポンチョ'],
    'コード': ['コード','cord'],
    'フレーム': ['frame','フレーム'],
    '携帯用品':['iphone'],
    'グラス': ['glass','グラス'],
    '靴下': ['靴下','socks','ソックス'],
    'ピッチャー': ['ヒッチ','ピッチャー']
}


query_keywords = sorted(query_keywords)
names = util.normalize_strings(query_keywords)
groups = util.get_groups(names,group_dict)
dictionary = { 'query_keywords': query_keywords, 'name': names, 'group': groups }
new_df = pd.DataFrame(dictionary)
new_df.head(5)

,group,name,query_keywords
0,ノブ,knob,KNOB
1,peg,peg,Peg
2,うちわ,うちわ,うちわ
3,財布,がまぐち,がまぐち
4,財布,がま口,がま口


### Export dictionaries as csv files

In [5]:
new_df.to_csv('../processed_data/subcategory.csv', header=True, mode='w')